In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../datasets/raw/movies.csv')
df.head()

,Title,Rating,Year,Month,Certificate,Runtime,Directors,Stars,Genre,Filming_location,Budget,Income,Country_of_origin
0,Avatar: The Way of Water,8.0,2022,December,PG-13,192,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...","Action, Adventure, Fantasy",New Zealand,"$350,000,000","$681,081,686",United States
1,Guillermo del Toro's Pinocchio,7.8,2022,December,PG,117,"Guillermo del Toro, Mark Gustafson","Ewan McGregor, David Bradley, Gregory Mann, Bu...","Animation, Drama, Family",USA,"$35,000,000","$71,614","United States, Mexico, France"
2,Bullet Train,7.3,2022,August,R,127,David Leitch,"Brad Pitt, Joey King, Aaron Taylor Johnson, Br...","Action, Comedy, Thriller",Japan,"$85,900,000","$239,268,602","Japan, United States"
3,The Banshees of Inisherin,8.0,2022,November,R,114,Martin McDonagh,"Colin Farrell, Brendan Gleeson, Kerry Condon, ...","Comedy, Drama",Ireland,Unknown,"$19,720,823","Ireland, United Kingdom, United States"
4,M3gan,NaN,2022,January,PG-13,102,Gerard Johnstone,"Jenna Davis, Amie Donald, Allison Williams, Vi...","Horror, Sci-Fi, Thriller",New Zealand,Unknown,Unknown,United States


In [3]:
df.columns = df.columns.str.lower()

In [4]:
memory_before = df.memory_usage(deep=True).sum()
memory_before

1622481

In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              2000 non-null   object 
 1   rating             1998 non-null   float64
 2   year               2000 non-null   int64  
 3   month              2000 non-null   object 
 4   certificate        1966 non-null   object 
 5   runtime            2000 non-null   object 
 6   directors          2000 non-null   object 
 7   stars              2000 non-null   object 
 8   genre              2000 non-null   object 
 9   filming_location   2000 non-null   object 
 10  budget             2000 non-null   object 
 11  income             2000 non-null   object 
 12  country_of_origin  2000 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.5 MB


In [6]:
df.fillna(
    {
        'certificate': df.certificate.mode()[0],
        'rating': df.rating.median()
    }, inplace=True
)

In [7]:
for column in df.columns:
    print(f"{column}: {len(df[column].unique())}")

title: 1989
rating: 62
year: 20
month: 14
certificate: 12
runtime: 113
directors: 1082
stars: 1990
genre: 244
filming_location: 97
budget: 305
income: 1855
country_of_origin: 406


In [8]:
df.year = df.year.astype('uint16')

In [9]:
df.month.unique()

array(['December', 'August', 'November', 'January', 'October', 'March',
       'September', 'May', 'April', 'July', 'June', 'February', '2014',
       '2008'], dtype=object)

In [10]:
df.loc[df.query('month == "2014" | month == "2008"').index, 'month'] = df.month.mode()[0]
df.month = df.month.astype('category')

In [11]:
df.certificate.unique()

array(['PG-13', 'PG', 'R', 'TV-14', 'TV-MA', 'TV-PG', 'TV-Y7',
       'Not Rated', 'NC-17', 'TV-G', 'Unrated', 'G'], dtype=object)

In [12]:
df.loc[(df.query('certificate == "Not Rated" | certificate == "Unrated"')).index, 'certificate'] = 'Unrated'
df.certificate = df.certificate.astype('category')

In [13]:
df.loc[(df.query('runtime == "Unknown"')).index, 'runtime'] = df.drop(index = df.query('runtime == "Unknown"').index).runtime.astype('int64').median()
df.runtime = df.runtime.astype('uint8')

In [14]:
df.filming_location = df.filming_location.astype('category')

In [15]:
df.budget.replace(to_replace=r'[,]', value='', regex=True, inplace=True)

In [16]:
df.income = df.income.str.lstrip('$').replace(to_replace=r'[,]', value='', regex=True)

In [17]:
a = df.income.unique()

In [18]:
df.budget.replace(to_replace = r'\d+', value= '', regex=True).unique()

array(['$', 'Unknown', '€', '₹', 'CA$', 'SEK\xa0', 'DKK\xa0', '£', '₩',
       '¥', 'A$', 'NOK\xa0', 'CN¥'], dtype=object)

In [19]:
df.loc[(df.query('income == "Unknown"')).index, 'income'] = df.drop(index = df.query('income == "Unknown"').index).income.astype('int64').median()

In [20]:
df.income = df.income.astype('uint32')
df.rename(columns={'income': 'income_$'}, inplace=True)

In [21]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   title              2000 non-null   object  
 1   rating             2000 non-null   float64 
 2   year               2000 non-null   uint16  
 3   month              2000 non-null   category
 4   certificate        2000 non-null   category
 5   runtime            2000 non-null   uint8   
 6   directors          2000 non-null   object  
 7   stars              2000 non-null   object  
 8   genre              2000 non-null   object  
 9   filming_location   2000 non-null   category
 10  budget             2000 non-null   object  
 11  income_$           2000 non-null   uint32  
 12  country_of_origin  2000 non-null   object  
dtypes: category(3), float64(1), object(6), uint16(1), uint32(1), uint8(1)
memory usage: 985.1 KB


In [22]:
memory_after = df.memory_usage(deep=True).sum()
memory_after

1008728

In [23]:
print (f'{memory_after/memory_before*100: .2f}% ganho de memória')

 62.17% ganho de memória


In [24]:
df.to_pickle('../datasets/processed/movies.pkl')